In [1]:
import os
os.chdir('/nfs/homedirs/ayle/guided-research/SNIP-it/')

In [2]:
import torch
from torchvision import datasets, transforms
import foolbox as fb
from experiments.main import load_checkpoint
from models import GeneralModel
from models.statistics.Metrics import Metrics
from utils.config_utils import *
from utils.model_utils import *
from utils.system_utils import *
from utils.attacks_utils import get_attack
from torch.utils.data.dataset import Dataset

In [3]:
arguments = dict({
'eval_freq': 1000,  # evaluate every n batches
    'save_freq': 1e6,  # save model every n epochs, besides before and after training
    'batch_size': 512,  # size of batches, for Imagenette 128
    'seed': 333,  # random seed
    'max_training_minutes': 6120 , # one hour and a 45 minutes max, process killed after n minutes (after finish of epoch)
    'plot_weights_freq': 50, # plot pictures to tensorboard every n epochs
    'prune_freq': 1, # if pruning during training: how long to wait before starting
    'prune_delay': 0, # "if pruning during training: 't' from algorithm box, interval between pruning events, default=0
    'prune_to': 10,
    'epochs': 80,
    'rewind_to': 0, # rewind to this epoch if rewinding is done
    'snip_steps': 5, # 's' in algorithm box, number of pruning steps for 'rule of thumb', TODO
    'pruning_rate': 0.0 , # pruning rate passed to criterion at pruning event. however, most override this
    'growing_rate': 0.0000 , # grow back so much every epoch (for future criterions)
    'pruning_limit': 0.00,  # Prune until here, if structured in nodes, if unstructured in weights. most criterions use this instead of the pruning_rate
    'learning_rate': 2e-3,
    'grad_clip': 10,
    'grad_noise': 0 , # added gaussian noise to gradients
    'l2_reg': 5e-5 , # weight decay
    'l1_reg': 0 , # l1-norm regularisation
    'lp_reg': 0 , # lp regularisation with p < 1
    'l0_reg': 1.0 , # l0 reg lambda hyperparam
    'hoyer_reg': 0.001 , # hoyer reg lambda hyperparam
    'beta_ema': 0.999 , # l0 reg beta ema hyperparam

    'loss': 'CrossEntropy',
    'optimizer': 'ADAM',
    'model': 'ResNet18',  # ResNet not supported with structured
    'data_set': 'CIFAR10',
    'ood_data_set': 'SVHN',
    'prune_criterion': 'EmptyCrit',  # options: SNIP, SNIPit, SNIPitDuring, UnstructuredRandom, GRASP, HoyerSquare, IMP, // SNAPit, StructuredRandom, GateDecorators, EfficientConvNets, GroupHoyerSquare
    'train_scheme': 'DefaultTrainer' , # default: DefaultTrainer
    'attack': 'FGSM',
    'epsilon': 48,
    'eval_ood_data_sets': ['SVHN', 'CIFAR100', 'GAUSSIAN', 'OODOMAIN'],
    'eval_attacks': ['FGSM', 'L2FGSM'],
    'eval_epsilons': [4, 6, 12, 24, 48],

    'device': 'cuda',
    'results_dir': "ResNet18",

    'checkpoint_name': None,
    'checkpoint_model': None,

    'disable_cuda_benchmark': 1 , # speedup (disable) vs reproducibility (leave it)
    'eval': 0,
    'disable_autoconfig': 0 , # for the brave
    'preload_all_data': 0 , # load all data into ram memory for speedups
    'tuning': 0 , # splits trainset into train and validationset, omits test set

    'track_weights': 0 , # "keep statistics on the weights through training
    'disable_masking': 1 , # disable the ability to prune unstructured
    'enable_rewinding': 0 , # enable the ability to rewind to previous weights
    'outer_layer_pruning': 1, # allow to prune outer layers (unstructured) or not (structured)
    'first_layer_dense': 0,
    'random_shuffle_labels': 0  ,# run with random-label experiment from zhang et al
    'l0': 0,  # run with l0 criterion, might overwrite some other arguments
    'hoyer_square': 0, # "run in unstructured DeephoyerSquare criterion, might overwrite some other arguments
    'group_hoyer_square': 0 ,# run in unstructured Group-DeephoyerSquare criterion, might overwrite some other arguments

    'disable_histograms': 0,
    'disable_saliency': 0,
    'disable_confusion': 0,
    'disable_weightplot': 0,
    'disable_netplot': 0,
    'skip_first_plot': 0,
    'disable_activations': 0,
    
#     'input_dim': [1, 28, 28],
#       'output_dim': 10,
#       'hidden_dim': [512],
#       'N': 60000,
    
    'input_dim': [3, 32, 32],
      'output_dim': 10,
      'hidden_dim': [512],
      'N': 60000
})

In [4]:
DATASET_PATH = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/data'

In [5]:
metrics = Metrics()
out = metrics.log_line
metrics._batch_size = arguments['batch_size']
metrics._eval_freq = arguments['eval_freq']
set_results_dir(arguments["results_dir"])

In [6]:
model: GeneralModel = find_right_model(
        NETWORKS_DIR, arguments['model'],
        device=arguments['device'],
        hidden_dim=arguments['hidden_dim'],
        input_dim=arguments['input_dim'],
        output_dim=arguments['output_dim'],
        is_maskable=arguments['disable_masking'],
        is_tracking_weights=arguments['track_weights'],
        is_rewindable=arguments['enable_rewinding'],
        is_growable=arguments['growing_rate'] > 0,
        outer_layer_pruning=arguments['outer_layer_pruning'],
        maintain_outer_mask_anyway=(
                                       not arguments['outer_layer_pruning']) and (
                                           "Structured" in arguments['prune_criterion']),
        l0=arguments['l0'],
        l0_reg=arguments['l0_reg'],
        N=arguments['N'],
        beta_ema=arguments['beta_ema'],
        l2_reg=arguments['l2_reg']
).to(arguments['device'])

Ignored arguments in ResNet18: {'hidden_dim': [512]}


In [7]:
load_checkpoint(arguments, model, out)

In [8]:
device = arguments['device']

In [9]:
# get criterion
criterion = find_right_model(
    CRITERION_DIR, arguments['prune_criterion'],
    model=model,
    limit=arguments['pruning_limit'],
    start=0.5,
    steps=arguments['snip_steps'],
    device=arguments['device'],
    arguments=arguments
)

# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments
)

# get loss function
loss = find_right_model(
    LOSS_DIR, arguments['loss'],
    device=device,
    l1_reg=arguments['l1_reg'],
    lp_reg=arguments['lp_reg'],
    l0_reg=arguments['l0_reg'],
    hoyer_reg=arguments['hoyer_reg']
)

# get optimizer
optimizer = find_right_model(
    OPTIMS, arguments['optimizer'],
    params=model.parameters(),
    lr=arguments['learning_rate'],
    weight_decay=arguments['l2_reg'] if not arguments['l0'] else 0
)

run_name = f'_model={arguments["model"]}_dataset={arguments["data_set"]}_ood-dataset={arguments["ood_data_set"]}' + \
           f'_attack={arguments["attack"]}_epsilon={arguments["epsilon"]}_prune-criterion={arguments["prune_criterion"]}' + \
           f'_pruning-limit={arguments["pruning_limit"]}_prune-freq={arguments["prune_freq"]}_prune-delay={arguments["prune_delay"]}' + \
           f'_rewind-to={arguments["rewind_to"]}_train-scheme={arguments["train_scheme"]}_seed={arguments["seed"]}'


# build trainer
trainer = find_right_model(
    TRAINERS_DIR, arguments['train_scheme'],
    model=model,
    loss=loss,
    optimizer=optimizer,
    device=device,
    arguments=arguments,
    train_loader=train_loader,
    test_loader=test_loader,
    ood_loader=ood_loader,
    metrics=metrics,
    criterion=criterion,
    run_name=run_name
)

trainer.train()

Ignored arguments in EmptyCrit: {'limit': 0.0, 'start': 0.5, 'steps': 5, 'arguments': {'eval_freq': 1000, 'save_freq': 1000000.0, 'batch_size': 512, 'seed': 333, 'max_training_minutes': 6120, 'plot_weights_freq': 50, 'prune_freq': 1, 'prune_delay': 0, 'prune_to': 10, 'epochs': 80, 'rewind_to': 0, 'snip_steps': 5, 'pruning_rate': 0.0, 'growing_rate': 0.0, 'pruning_limit': 0.0, 'learning_rate': 0.002, 'grad_clip': 10, 'grad_noise': 0, 'l2_reg': 5e-05, 'l1_reg': 0, 'lp_reg': 0, 'l0_reg': 1.0, 'hoyer_reg': 0.001, 'beta_ema': 0.999, 'loss': 'CrossEntropy', 'optimizer': 'ADAM', 'model': 'ResNet18', 'data_set': 'CIFAR10', 'ood_data_set': 'SVHN', 'prune_criterion': 'EmptyCrit', 'train_scheme': 'DefaultTrainer', 'attack': 'FGSM', 'epsilon': 48, 'eval_ood_data_sets': ['SVHN', 'CIFAR100', 'GAUSSIAN', 'OODOMAIN'], 'eval_attacks': ['FGSM', 'L2FGSM'], 'eval_epsilons': [4, 6, 12, 24, 48], 'device': 'cuda', 'results_dir': 'ResNet18', 'checkpoint_name': None, 'checkpoint_model': None, 'disable_cuda_ben

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.7734375  |  0.6374625   |    0.7177279    |  0.9082095  | 0.6861615  |   0.7934825    |        0.9990234        |   1.1542673   |    0.7121359    |   0.5821396    |    3505696.0     |    0.0000000   
$ |  sparse/node  |  sparse/hm  |  sparse/log_disk_size  |  time/gpu_time  |  time/flops_per_sample  |  time/flops_log_cum  |  cuda/ram_footprint  |  time/batch_time  |  
$ |   0.0000000   |  0.0000000  |       19.8013823       |   111.5923155   |       38800136.0        |      12.8924958      |     241325056.0      |     0.0286453     |
Training... 97/98


> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.8945312  |  0.3079255   |    0.3609815    |  0.8069188  | 0.7392348  |   0.4782842    |        0.9975586        |   0.7468569   |    0.7125941    |   0.6285410    |    3173582.0     |    0.0000000   
$ |  sparse/node  |  sparse/hm  |  sparse/log_disk_size  |  time/gpu_time  |  time/flops_per_samp

Training... 97/98



EPOCH 16  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9355469  |  0.1967722   |    0.2210181    |  0.8461214  | 0.7555894  |   0.3434358    |        0.9988281        |   0.6484223   |    0.

Training... 97/98



EPOCH 22  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9570312  |  0.1310249   |    0.1641658    |  0.8510990  | 0.7711282  |   0.2901545    |        0.9982537        |   0.7022069   |    0.

...

Training... 97/98



EPOCH 58  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9707031  |  0.0843949   |    0.0815668    |  0.8671770  | 0.7892521  |   0.2212998    |        0.9956170        |   0.5243995   |    0.

Training... 97/98



EPOCH 64  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9843750  |  0.0479666   |    0.0658917    |  0.9352181  | 0.7841567  |   0.2085288    |        0.9935777        |   0.4191877   |    0.

Training... 97/98



EPOCH 70  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9785156  |  0.0642308   |    0.0773380    |  0.9404792  | 0.7820830  |   0.2096304    |        0.9959099        |   0.4244646   |    0.

Training... 97/98



EPOCH 76  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9785156  |  0.0754224   |    0.0654612    |  0.9300239  | 0.7817727  |   0.2047769    |        0.9932732        |   0.4969049   |    0.

<Figure size 432x288 with 0 Axes>

In [10]:
results = {'train_acc': trainer.train_acc, 'test_acc': trainer.test_acc, 'sparsity': trainer.sparsity,
               'filename': DATA_MANAGER.stamp}

In [19]:
model.eval()

ResNet18(
  (conv1): ContainerConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.05, inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): ContainerConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.05, inplace=True)
      (conv2): ContainerConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): ContainerConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [13]:
for attack in arguments['eval_attacks']:
    for epsilon in arguments['eval_epsilons']:
        out("Attack {}".format(attack))
        # build tester
        tester = find_right_model(
            TESTERS_DIR, 'AdversarialEvaluation',
            attack=attack,
#             model=trainer._model,
            model = model,
            device=device,
            arguments=None,
            test_loader=test_loader,
        )

        out("Epsilon {}".format(str(epsilon)))
        res = tester.evaluate(epsilon=epsilon)

        for key, value in res.items():
            results[key] = value

Attack FGSM
Epsilon 4
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.4765625 same prediction as model 0.720703125 bounds adver -2.1179039478302 2.640000104904175 norm 0.8684147793101147
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.435546875 same prediction as model 0.763671875 bounds adver -2.1179039478302 2.640000104904175 norm 0.868644886650145
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.451171875 same prediction as model 0.73828125 bounds adver -2.1179039478302 2.640000104904175 norm 0.8684394153533503
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.439453125 same prediction as model 0.76953125 bounds adver -2.1179039478302 2.640000104904175 norm 0.8684375159209594
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.474609375 same prediction as model 0.732421875 bounds adver -2.1179039478302 2.640000104904175 norm 0.8687101391842589
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.48828125 same prediction as model 0.7304

EPSILON 12 Successes attack 0.759765625 same prediction as model 0.453125 bounds adver -2.1179039478302 2.640000104904175 norm 2.6034983382560313
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.736328125 same prediction as model 0.43359375 bounds adver -2.1179039478302 2.640000104904175 norm 2.6040616505779326
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.748046875 same prediction as model 0.46875 bounds adver -2.1179039478302 2.640000104904175 norm 2.603162937797606
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.787109375 same prediction as model 0.42578125 bounds adver -2.1179039478302 2.640000104904175 norm 2.6037429105490446
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.78515625 same prediction as model 0.423828125 bounds adver -2.1179039478302 2.640000104904175 norm 2.602934836409986
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.796875 same prediction as model 0.41015625 bounds adver -2.1179039478302 2.640000104904175 n

EPSILON 48 Successes attack 0.986328125 same prediction as model 0.2265625 bounds adver -2.1179039478302 2.640000104904175 norm 10.392164763063192
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.99609375 same prediction as model 0.216796875 bounds adver -2.1179039478302 2.640000104904175 norm 10.393288457766175
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.990234375 same prediction as model 0.201171875 bounds adver -2.1179039478302 2.640000104904175 norm 10.390885325148702
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.984375 same prediction as model 0.23828125 bounds adver -2.1179039478302 2.640000104904175 norm 10.385771203786135
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.98828125 same prediction as model 0.193359375 bounds adver -2.1179039478302 2.640000104904175 norm 10.390090199187398
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.986328125 same prediction as model 0.203125 bounds adver -2.1179039478302 2.640000104904

EPSILON 6 Successes attack 0.25735294818878174 same prediction as model 0.9926470518112183 bounds adver -2.1179039478302 2.640000104904175 norm 0.023512461907504237
Attack L2FGSM
Epsilon 12
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.228515625 same prediction as model 0.97265625 bounds adver -2.1179039478302 2.640000104904175 norm 0.047030608235218097
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.220703125 same prediction as model 0.982421875 bounds adver -2.1179039478302 2.640000104904175 norm 0.04703753905778285
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.208984375 same prediction as model 0.984375 bounds adver -2.1179039478302 2.640000104904175 norm 0.04703358931874391
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.236328125 same prediction as model 0.978515625 bounds adver -2.1179039478302 2.640000104904175 norm 0.047034040035214275
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.23046875 same prediction as

EPSILON 48 Successes attack 0.310546875 same prediction as model 0.91015625 bounds adver -2.1179039478302 2.640000104904175 norm 0.18810954873333685
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.29296875 same prediction as model 0.92578125 bounds adver -2.1179039478302 2.640000104904175 norm 0.18812476584571414
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.240234375 same prediction as model 0.9296875 bounds adver -2.1179039478302 2.640000104904175 norm 0.1881488137296401
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.302734375 same prediction as model 0.919921875 bounds adver -2.1179039478302 2.640000104904175 norm 0.18812019797042012
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.302734375 same prediction as model 0.916015625 bounds adver -2.1179039478302 2.640000104904175 norm 0.18813256136490963
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.279296875 same prediction as model 0.931640625 bounds adver -2.11790394

In [20]:
with torch.no_grad():
    for ood_data_set in arguments['eval_ood_data_sets']:
        out("OOD Dataset: {}".format(ood_data_set))
        # load data
        _, test_loader = find_right_model(
            DATASETS, arguments['data_set'],
            arguments=arguments
        )

        # load OOD data
        _, ood_loader = find_right_model(
            DATASETS, ood_data_set,
            arguments=arguments
        )
        # build tester
        tester = find_right_model(
            TESTERS_DIR, 'OODEvaluation',
#             model=trainer._model,
            model = model,
            device=device,
            arguments=None,
            test_loader=test_loader,
            ood_loader=ood_loader,
            ood_dataset=ood_data_set
        )
        res = tester.evaluate()

        for key, value in res.items():
            results[key] = value
            
        break

OOD Dataset: SVHN
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: gitignored/data/train_32x32.mat
Using downloaded and verified file: gitignored/data/test_32x32.mat


In [17]:
class DS(Dataset):

    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.mean = [0.485, 0.456, 0.406]  # avg 0.449
        self.std = [0.229, 0.224, 0.225]  # avg 0.226
        self.transforms = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ]
        )

    def __getitem__(self, item):
        image = self.images[item] / 255
        image = self.transforms(image.transpose((1, 2, 0)))
        return image.to(torch.float32), torch.tensor(self.labels[item], dtype=torch.float32)

    def __len__(self):
        return len(self.images)

with torch.no_grad():
    if arguments["data_set"] == "CIFAR10":
        ds_path = os.path.join(DATASET_PATH, "cifar10_corrupted")
        for ds_dataset_name in os.listdir(ds_path):
            npz_dataset = np.load(os.path.join(ds_path, ds_dataset_name))

            ds_dataset = DS(npz_dataset["images"], npz_dataset["labels"])
            ds_loader = torch.utils.data.DataLoader(
                ds_dataset,
                batch_size=arguments['batch_size'],
                shuffle=False,
                pin_memory=True,
                num_workers=4
            )

            # build tester
            tester = find_right_model(
                TESTERS_DIR, 'DSEvaluation',
                model=trainer._model,
                device=device,
                arguments=None,
                test_loader=test_loader,
                ds_loader=ds_loader,
                ds_dataset=ds_dataset_name.split('.')[0]
            )
            res = tester.evaluate()

            for key, value in res.items():
                results[key] = value

            break

In [21]:
results

{'success_rate_FGSM_4': 0.4614430144429207,
 'auroc_FGSM_4': 0.585112205,
 'aupr_FGSM_4': 0.6009618942917225,
 'entropy_FGSM_4': 0.24559109,
 'success_rate_FGSM_6': 0.5628331810235977,
 'auroc_FGSM_6': 0.59242727,
 'aupr_FGSM_6': 0.6192600772900909,
 'entropy_FGSM_6': 0.24968901,
 'success_rate_FGSM_12': 0.7653779864311219,
 'auroc_FGSM_12': 0.5732391050000001,
 'aupr_FGSM_12': 0.5903260204166738,
 'entropy_FGSM_12': 0.25529772,
 'success_rate_FGSM_24': 0.9361098349094391,
 'auroc_FGSM_24': 0.5170752200000001,
 'aupr_FGSM_24': 0.5017410279933904,
 'entropy_FGSM_24': 0.24315119,
 'success_rate_FGSM_48': 0.9897690713405609,
 'auroc_FGSM_48': 0.48606827999999996,
 'aupr_FGSM_48': 0.4646806659687912,
 'entropy_FGSM_48': 0.23248205,
 'success_rate_L2FGSM_4': 0.2190199911594391,
 'auroc_L2FGSM_4': 0.503650815,
 'aupr_L2FGSM_4': 0.503327894466636,
 'entropy_L2FGSM_4': 0.20600477,
 'success_rate_L2FGSM_6': 0.2224379599094391,
 'auroc_L2FGSM_6': 0.5054435500000001,
 'aupr_L2FGSM_6': 0.504928387